In [1]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker
from scipy import optimize
from scipy.integrate import simps 
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.optimize import curve_fit


from scipy.optimize import curve_fit


from matplotlib import rc
plt.rc('text', usetex=True)
plt.rc('font', family='serif')


def fun_para():
    # vg = 10*1e-06 #m/sn=2
    # c_inf=1.*1000 #mol/m3
    dV = 1. #V
    Dc=4.5e-010 #m2/s
    Da=4.5e-010 #m2/s

    RR=8.314 #J/mol/K
    TT=273.15+20 #K
    uc=Dc/(RR*TT)
    ua=Da/(RR*TT)
    tc = 0.393 #uc/(uc+ua)

    n=2.
    F=96500.
    rho=8.95*1000 #kg/m3
    gamma = 1286*0.001 #1148.81*0.001 #J/m2
    M = 63.546*0.001 #kg/mol
    vm = M/rho
    epss=8.854*1e-012 #C2/N/m2
    epss0=78.54


    return dV, Dc, Da, tc, n, F, rho, gamma, M, vm, epss, epss0, RR, TT, uc, ua



[dV, Dc, Da, tc, n, F, rho, gamma, M, vm, epss, epss0, RR, TT, uc, ua] = fun_para()




def fun_rI(x,j00,c00,R):
    # j00 : current density (A/m2)
    # c00 : concentration (mol/m3)
    # R : particle radius (m)
    vgg = (1-tc)*j00/(n*F*c00)
    jgg = (1./3.)*n*F*(rho/M)*vgg
    return dV-(-R)*((R*2*jgg/(epss*epss0*n*F*uc)))**0.5*((1-x)**0.5-(1./x)**0.5*np.arctan(((1-x)/x)**0.5))


def fun_rI2(x,R):
    D=-(2./3.)*F*n/(epss*epss0)
    return x**3-(3./2.)*R*x**2+0.5*R**3+dV*R/D

def fun_rI3(x,a,b,c):
    return a*x**(3/2.)-b*x**0.5-c

def fun_rI4(x,dV,A,R):
    r = np.arange(R,x,(x-R)/1000.)
    y = (x**3-r**3)**0.5/r**2
    return dV-A*simps(y,r)


def func_adjust_Rg075(x,a0,N):
    return a0*(92.3/x)**N

def func_adjust_Rg05(x,a0,N):
    return a0*(82.1/x)**N

# calcul de xI numérique

In [3]:
tau = gamma*vm/(RR*TT)

print((6*tau*0.1*n*F/(gamma*vm))*exp(-n*F*0.1/(RR*TT)))

print(6*tau*0.1*exp(-n*F*0.1/(RR*TT)))
print(gamma*vm/(n*F*0.1))

print(6*tau*1e+09)

print('-----')
vg = (1-tc)*80*10/(n*F*500)
jg = n*F*(rho/M)*vg
Ig = 4*pi*(330*1e-09)**2*jg
tt = ((22.5*1e-09)**3-(0.5*1e-09)**3)/(3*Ig/(4*pi*(rho/M)*n*F))

print(Ig)
print(tt)
print(2*330*1e-09/vg)
print(tt/(2*330*1e-09/vg))


print('-----')


print (gamma*vm/(RR*TT))

print ((2*4*80*1e-09*gamma*vm/(RR*TT))**0.5)


print ('----')
print(gamma*vm/(n*F*0.1))

print(6*gamma*vm*np.exp(-n*F*0.01/(RR*TT))/(RR*TT))



0.017287540438392566
8.178658068612319e-13
4.730955278316496e-10
22.477990502860386
-----
1.8718939231022345e-07
6.928548943581893e-06
0.13115733113673805
5.282624222018162e-05
-----
3.7463317504767314e-09
4.896582808760726e-08
----
4.730955278316496e-10
1.0182394996329245e-08


In [18]:
#j = np.array([5., 15, 30., 50., 60., 70., 80., 90., 100.])*10

j_c0 = 80.*10/500
c00 = 500.
#R = 350*1e-09*80*10/j00
l = 2.
R = 333.89*1e-09

vg = (1-tc)*j_c0/(n*F)
jg = (1./6.)*n*F*(rho/M)*vg
A = ((2./3.)*jg*F*n/(n*F*n*uc*F*epss*epss0))**0.5*R

sol = optimize.fsolve(fun_rI4,R*1.05,args=(dV,A,R))

rI = sol[0]
xI = rI-R


xI_chaz = ((3./2.)*dV)**(2/3.)*(epss*epss0*n*F*uc/(2*jg))**(1/3.)  
c_s_chaz = (jg*epss*epss0/(2*n*F*n*uc*F*F*n*xI_chaz))**0.5

c_s = (3*jg*epss*epss0/(2*n*F*n*uc*F*F*n))**0.5*(R**2/(rI**3-R**3))**0.5

eta0 = (RR*TT/(n*F))*np.log(c_s/c00)
eta0_chaz = (RR*TT/(n*F))*np.log(c_s_chaz/c00)


print ('xI=', xI*1e+09)
print ('x_I Chazalviel=',xI_chaz*1e+09)
print ('----------')
print ('c_s =', c_s)
print ('c_s_chaz =', c_s_chaz)
print ('--------------')
print ('eta0 =', -eta0)
print ('eta0_chaz =', -eta0_chaz)
print ('---------')
print ('R_c =', (2*gamma*vm/(RR*TT)+(2*gamma*vm/(RR*TT)*xI*(l+2)*(l-1))**0.5)*1e+09)
print ('---------')
print ('R_t =', (xI + 2*gamma*vm/(RR*TT)*(l+2)*(l-1))*1e+09)

print(jg*4*np.pi*(333.89*1e-09)**2)
 
#xI : [320.517584, 220.4172329, 174.134771, 146.41503172, 137.638260 , 130.632996, 124.856198, 119.97502, 115.772043 ]
#c_s : [0.01850484, 0.0433109, 0.07292425, 0.10634875, 0.121528, 0.1359761, 0.149821, 0.163158247, 0.176056 ]
# eta0 : [0.1339829, 0.1232442, 0.1166647, 0.1119001, 0.11021518, 0.10879664, 0.10757213, 0.106495, 0.10553442 ]

xI= 108.84689046791844
x_I Chazalviel= 106.93508039873417
----------
c_s = 0.200747371717312
c_s_chaz = 0.23631645785347433
--------------
eta0 = 0.09875677936697413
eta0_chaz = 0.09669680418489562
---------
R_c = 64.60844497687312
---------
R_t = 138.8175444717323
3.1938089129176564e-08


In [14]:
print 750./0.16
print Dc*np.log(1.+(np.exp(1.)-1)*750./0.16)/((1-0.39)*(ua+uc)*2*96500)

print Dc*np.log(1.+(np.exp(1.)-1)*750./0.33)/((1-0.39)*(ua+uc)*2*96500)

print Dc*np.log(1.+(np.exp(1.)-1)*250./0.33)/((1-0.39)*(ua+uc)*2*96500)

print Dc*np.log(1.+(np.exp(1.)-1)*250./0.7)/((1-0.39)*(ua+uc)*2*96500)

4687.5
0.09309806795385465
0.08560614112358225
0.07423969145413167
0.06646476345593587


In [55]:
j_ex = 1. #1-100 A/m2
etaa = 0.1

print 1.38*1e-023*TT*gamma/(2*1.6*1e-019*j_ex*etaa**2)

#0.0163 - 1.626 sec

print 550*1e-09/(1e-06)

#0.0075 - 0.55


1.62577325625
0.55
0.460122699387


# plot xI vs xI chazalviel

In [4]:
figure(figsize=(9,8))

plot([25, 50, 100,150, 250, 350, 450, 550],[90.11, 88.72, 87.24, 86.49, 85.71, 85.32, 85.09, 84.93],'-ro',label=r'Spherical model (Eq. 11), $c_0$ = 0.50 M, $j=j_c$(0.50 M) = 82.1 mA/cm$^2$')
plot([150, 250, 350, 450, 550],[95.4, 94.5, 94.05, 93.77, 93.58],'-bs',label=r'Spherical model (Eq. 11), $c_0$ = 0.75 M, $j=j_c$(0.75 M) = 92.3 mA/cm$^2$')

plot([150, 550],[84.14, 84.14],'--k',label=r'1D model, $c_0$ = 0.50 M, $j=j_c$(0.50 M) = 82.1 mA/cm$^2$')
plot([150, 550],[92.63, 92.63],'-k',label=r'1D model, $c_0$ = 0.75 M, $j=j_c$(0.75 M) = 92.3 mA/cm$^2$')


xlabel(r'$R_g$ (nm)',fontsize=20)
ylabel(r'$x_I(T)$ (nm)',fontsize=20)

xticks(fontsize=15)
yticks(fontsize=15)

legend(loc='upper right',fontsize=16) 

xlim(20,600)
ylim(80,100)

(80, 100)

# plot cs vs cs chazalviel

In [5]:
figure(figsize=(9,8))

xI__05 = np.array([90.11, 88.72, 87.24, 86.49, 85.71, 85.32, 85.09, 84.93])
R__ = np.array([25, 50, 100,150, 250, 350, 450, 550])
cs__05 = np.array([0.123, 0.190, 0.257, 0.290, 0.322, 0.337, 0.346, 0.352])


xI__075 = np.array([99.39, 97.90, 96.26, 95.4, 94.5, 94.05, 93.77, 93.58])

cs__075 = np.array([0.095, 0.149, 0.205, 0.233, 0.261, 0.275, 0.283, 0.288])



xI__05_jc2 = np.array([114.03, 112.40, 110.52, 109.49, 108.40, 107.82, 107.47, 107.23])

cs__05_jc2 = np.array([0.066, 0.105, 0.149, 0.171, 0.194, 0.206, 0.213, 0.218])


plot(xI__05/R__,np.log(1-(-cs__05+0.382)/0.382),'-ro',label=r'Spherical model (Eq. 11), $c_0$ = 0.50 M, $j=j_c$(0.50 M) = 82.1 mA/cm$^2$')
plot(xI__075/R__,np.log(1-(-cs__075+0.315)/0.315),'-bs',label=r'Spherical model (Eq. 11), $c_0$ = 0.75 M, $j=j_c$(0.75 M) = 82.1 mA/cm$^2$')
plot(xI__05_jc2/R__,np.log(1-(-cs__05_jc2+0.240)/0.240),'-go',label=r'Spherical model (Eq. 11), $c_0$ = 0.50 M, $j=j_c$(0.50 M) = 82.1 mA/cm$^2$')

y = np.concatenate((np.log(1-(-cs__05[1:-1]+0.382)/0.382), np.log(1-(-cs__075[1:-1]+0.315)/0.315), np.log(1-(-cs__05_jc2[1:-1]+0.240)/0.240)),axis=0)
x = np.concatenate((xI__05[1:-1]/R__[1:-1], xI__075[1:-1]/R__[1:-1], xI__05_jc2[1:-1]/R__[1:-1] ))
print (x)
figure()
plot(x,y,'.')

p = np.polyfit(x,y,1)
print (p)

plot(x,np.polyval(p,x),'r')
              
              

figure()
plot((xI__05/R__)**1,((-cs__05+0.382)/0.382),'-ro',label=r'Spherical model (Eq. 11), $c_0$ = 0.50 M, $j=j_c$(0.50 M) = 82.1 mA/cm$^2$')
plot((xI__075/R__)**1,((-cs__075+0.315)/0.315),'-bs',label=r'Spherical model (Eq. 11), $c_0$ = 0.75 M, $j=j_c$(0.75 M) = 82.1 mA/cm$^2$')
plot((xI__05_jc2/R__)**1,((-cs__05_jc2+0.240)/0.240),'-go',label=r'Spherical model (Eq. 11), $c_0$ = 0.50 M, $j=j_c$(0.50 M) = 82.1 mA/cm$^2$')


#plot([150, 250, 350, 450, 550],[95.4, 94.5, 94.05, 93.77, 93.58],'-bs',label=r'Spherical model (Eq. 11), $c_0$ = 0.75 M, $j=j_c$(0.75 M) = 92.3 mA/cm$^2$')

#plot([min(xI__05/R__), max(xI__05/R__)],np.array([0.382, 0.382]),'--k',label=r'1D model, $c_0$ = 0.50 M, $j=j_c$(0.50 M) = 82.1 mA/cm$^2$')
#plot([150, 550],[92.63, 92.63],'-k',label=r'1D model, $c_0$ = 0.75 M, $j=j_c$(0.75 M) = 92.3 mA/cm$^2$')


xlabel(r'$xI/R_g$ ',fontsize=20)
ylabel(r'$c_s$ (mol/m$^3$)',fontsize=20)

xticks(fontsize=15)
yticks(fontsize=15)

#legend(loc='upper right',fontsize=16) 

#xlim(20,600)
#ylim(80,100)

[1.7744     0.8724     0.5766     0.34284    0.24377143 0.18908889
 1.958      0.9626     0.636      0.378      0.26871429 0.20837778
 2.248      1.1052     0.72993333 0.4336     0.30805714 0.23882222]
[-0.36155092 -0.05176252]


(array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]),
 <a list of 9 Text yticklabel objects>)

# plot xI vs xI_chaz ) f(R, j/c0)

In [6]:
j_c0 = np.array([0.4, 3.05, 5.7, 8.35, 11.])

R_ = np.array([25., 50., 100., 150., 250., 350., 450., 550])

xI_ = np.zeros((5,8))
cs_ = np.zeros((5,8))

#xI_chaz = np.zeros((5,8))
#cs_chaz = np.zeros((5,8))


xI_[0,:]=[183.9277, 181.82325, 179.016118, 177.2479631,175.162941, 173.98023, 173.2202, 172.6912]
xI_[1,:]=[92.40749155, 90.9873020, 89.47263, 88.68579,  87.88332, 87.4772871, 87.232390, 87.06865]
xI_[2,:]=[74.698693, 73.4924, 72.29800, 71.71142, 71.13476, 70.85074, 70.68183, 70.569869]
xI_[3,:]=[65.58838, 64.508225, 63.48613,   63.000248, 62.53221, 62.30503, 62.17093, 62.08244]
xI_[4,:]=[59.7058, 58.713653, 57.8047, 57.38231, 56.98094,  56.787999, 56.67466, 56.60010]

cs_[0,:]=[0.0175348, 0.0301306, 0.046491, 0.05640911, 0.06761, 0.073709, 0.07751,  0.0801179]
cs_[1,:]=[0.115400, 0.178571, 0.24261, 0.27415, 0.304970, 0.3200110, 0.3288999, 0.33476]
cs_[2,:]=[0.202231, 0.30124162, 0.393831, 0.43689, 0.47753, 0.49690, 0.508229, 0.515649]
cs_[3,:]=[0.28336221, 0.412086, 0.526754, 0.578377,  0.626203, 0.648727, 0.661807, 0.67035084]
cs_[4,:]=[0.3605, 0.51525263,  0.6483938, 0.707019, 0.760674,  0.7857398,  0.800240, 0.80968849]


# anciennes versions (1/3)
#xI_[0,:]=[145.50, 143.62, 141.28, 139.91, 138.37, 137.53, 137.01, 136.64]
#xI_[1,:]=[73.00, 71.81, 70.65,70.08, 69.52, 69.25,  69.09, 68.98]
#xI_[2,:]=[58.98, 58.00, 57.11, 56.69, 56.30, 56.11, 56.00, 55.93]
#xI_[3,:]=[51.78, 50.91, 50.16, 49.82, 49.50,  49.35, 49.26, 49.21]
#xI_[4,:]=[47.13, 46.34, 45.68, 45.39, 45.12, 44.99, 44.92, 44.87]
#cs_[0,:]=[0.0337, 0.0560, 0.0829, 0.09799, 0.1142, 0.1226, 0.128, 0.1312]
#cs_[1,:]=[0.2147, 0.3185, 0.4148, 0.4596, 0.5012, 0.5211, 0.5327, 0.5403]
#cs_[2,:]=[0.3719, 0.5303, 0.666, 0.7256, 0.7800, 0.8054, 0.8201, 0.8297]
#cs_[3,:]=[0.5171, 0.7196, 0.8853, 0.9560, 1.0196,  1.0490, 1.0660, 1.0770]
#cs_[4,:]=[0.6543, 0.8947, 1.0852, 1.1650, 1.2360, 1.2686, 1.2873, 1.299]


vg_ = (1-tc)*j_c0/(n*F)
jg_ = (1./6.)*n*F*(rho/M)*vg_
xI_chaz_ = 1e+09*((3./2.)*dV)**(2/3.)*(epss*epss0*n*F*uc/(2*jg_))**(1/3.)  
c_s_chaz_ = (jg_*epss*epss0/(2*n*F*n*uc*F*F*n*1e-09*xI_chaz_))**0.5







figure(figsize=(9,8))
plot(R_[1:],xI_[0,1:],'.-b')
plot(R_[1:],xI_[1,1:],'.-b')
plot(R_[1:],xI_[2,1:],'.-b')
plot(R_[1:],xI_[3,1:],'.-b')
plot(R_[1:],xI_[4,1:],'.-b')

plot(R_[1:],xI_[4,1:],'.-b',label=r'$x_I(T)$ spherical model')



plot(R_[1:],R_[1:]*0+xI_chaz_[0],'--r')
plot(R_[1:],R_[1:]*0+xI_chaz_[1],'--r')
plot(R_[1:],R_[1:]*0+xI_chaz_[2],'--r')
plot(R_[1:],R_[1:]*0+xI_chaz_[3],'--r')
plot(R_[1:],R_[1:]*0+xI_chaz_[4],'--r')

plot(R_[1:]+5000,R_[1:]*0+xI_chaz_[4],'--r',label=r'$x_I^{1\mathrm{D}}$ one-dimensional model')

xlim(20,650)

xlabel(r'$R$ (nm)',fontsize=20)
ylabel(r'Thickness of space charge region  (nm)',fontsize=20)
xticks(fontsize=15)
yticks(fontsize=15)
legend(loc='center right',fontsize=18) 



figure(figsize=(9,8))
plot(xI_[0,1:]/R_[1:],xI_[0,1:]/xI_chaz_[0],'.r',label=r'$j/c_0$ = 0.40 A.m/mol')
plot(xI_[1,1:]/R_[1:],xI_[1,1:]/xI_chaz_[1],'sr',label=r'$j/c_0$ = 3.05 A.m/mol')
plot(xI_[2,1:]/R_[1:],xI_[2,1:]/xI_chaz_[2],'ok',label=r'$j/c_0$ = 5.70 A.m/mol')
plot(xI_[3,1:]/R_[1:],xI_[3,1:]/xI_chaz_[3],'vm',label=r'$j/c_0$ = 8.35 A.m/mol')
plot(xI_[4,1:]/R_[1:],xI_[4,1:]/xI_chaz_[4],'xg',label=r'$j/c_0$ = 11.0 A.m/mol')
#xlim(20,600)
ylim(0,1.2)

xlabel(r'$x_I(T)/R$',fontsize=20)
ylabel(r'$x_I(T)/x_I^{1\mathrm{D}}$',fontsize=20)

xticks(fontsize=15)
yticks(fontsize=15)

legend(loc='lower right',fontsize=18) 







In [24]:
figure()
plot(R_,cs_[0,:],'s')
plot(R_,cs_[1,:],'s')
plot(R_,cs_[2,:],'s')
plot(R_,cs_[3,:],'s')
plot(R_,cs_[4,:],'s')

plot(R_,R_*0+c_s_chaz_[0],'--r')
plot(R_,R_*0+c_s_chaz_[1],'--r')
plot(R_,R_*0+c_s_chaz_[2],'--r')
plot(R_,R_*0+c_s_chaz_[3],'--r')
plot(R_,R_*0+c_s_chaz_[4],'--r')

figure()
plot(xI_[0,:]/R_,cs_[0,:]/c_s_chaz_[0],'s')
plot(xI_[1,:]/R_,cs_[1,:]/c_s_chaz_[1],'s')
plot(xI_[2,:]/R_,cs_[2,:]/c_s_chaz_[2],'s')
plot(xI_[3,:]/R_,cs_[3,:]/c_s_chaz_[3],'s')
plot(xI_[4,:]/R_,cs_[4,:]/c_s_chaz_[4],'s')
#xlim(20,600)
ylim(0,1.2)

figure()
plot(xI_[0,:]/R_,np.log(cs_[0,:]/c_s_chaz_[0]),'s')
plot(xI_[1,:]/R_,np.log(cs_[1,:]/c_s_chaz_[1]),'s')
plot(xI_[2,:]/R_,np.log(cs_[2,:]/c_s_chaz_[2]),'s')
plot(xI_[3,:]/R_,np.log(cs_[3,:]/c_s_chaz_[3]),'s')
plot(xI_[4,:]/R_,np.log(cs_[4,:]/c_s_chaz_[4]),'s')
#xlim(20,600)
#ylim(0,1.2)


In [7]:
figure(figsize=(9,8))
plot(R_[1:],cs_[0,1:],'.-b')
plot(R_[1:],cs_[1,1:],'.-b')
plot(R_[1:],cs_[2,1:],'.-b')
plot(R_[1:],cs_[3,1:],'.-b')
plot(R_[1:],cs_[4,1:],'.-b',label=r'$c_s^+(T)$ spherical model')


plot(R_[1:],R_[1:]*0+c_s_chaz_[0],'--r')
plot(R_[1:],R_[1:]*0+c_s_chaz_[1],'--r')
plot(R_[1:],R_[1:]*0+c_s_chaz_[2],'--r')
plot(R_[1:],R_[1:]*0+c_s_chaz_[3],'--r')
plot(R_[1:],R_[1:]*0+c_s_chaz_[4],'--r',label=r'${c_s^+}^{1\mathrm{D}}$ one-dimensional model')

ylim(0,1.12)
xlim(20,650)

xlabel(r'$R$ (nm)',fontsize=20)
ylabel(r'Concentration of cations on the particle surface (mol/m$^3$)',fontsize=20)

xticks(fontsize=15)
yticks(fontsize=15)

legend(loc='upper center',fontsize=18) 



figure(figsize=(9,8))


y = np.concatenate((np.log(cs_[0,1:]/c_s_chaz_[0]), np.log(cs_[1,1:]/c_s_chaz_[1]), np.log(cs_[2,1:]/c_s_chaz_[2]), np.log(cs_[3,1:]/c_s_chaz_[3]), np.log(cs_[4,1:]/c_s_chaz_[4])),axis=0)
x = np.concatenate((xI_[0,1:]/R_[1:], xI_[1,1:]/R_[1:], xI_[2,1:]/R_[1:], xI_[3,1:]/R_[1:], xI_[4,1:]/R_[1:]))

p = np.polyfit(x,y,1)
print (p)

plot(x,np.polyval(p,x),'r')

semilogy(xI_[0,1:]/R_[1:],cs_[0,1:]/c_s_chaz_[0],'.r',label=r'$j/c_0$ = 0.40 A.m/mol')
semilogy(xI_[1,1:]/R_[1:],cs_[1,1:]/c_s_chaz_[1],'sr',label=r'$j/c_0$ = 3.05 A.m/mol')
semilogy(xI_[2,1:]/R_[1:],cs_[2,1:]/c_s_chaz_[2],'ok',label=r'$j/c_0$ = 5.70 A.m/mol')
semilogy(xI_[3,1:]/R_[1:],cs_[3,1:]/c_s_chaz_[3],'vm',label=r'$j/c_0$ = 8.35 A.m/mol')
semilogy(xI_[4,1:]/R_[1:],cs_[4,1:]/c_s_chaz_[4],'xg',label=r'$j/c_0$ = 11.0 A.m/mol')
xx = np.array([0,3.2])
yy = np.exp(np.polyval(p,xx))
semilogy(xx,yy,'r',label=r'linear fit, slope = 0.336')
#yscale('log')

xlabel(r'$x_I(T)/R$',fontsize=20)
ylabel(r'$c_s^+(T)/{c_s^+}^{1\mathrm{D}}$',fontsize=20)



xticks(fontsize=20)
yticks(fontsize=20)



legend(loc='upper right',fontsize=18) 


#xlim(20,650)
#ylim(0,1.2)







[-0.33585666 -0.05760673]


/home/fabien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: UserWarning: Data has no positive values, and therefore cannot be log-scaled.


In [12]:
#y = np.concatenate((np.log(cs_[0,:]/c_s_chaz_[0]), np.log(cs_[1,:]/c_s_chaz_[1]), np.log(cs_[2,:]/c_s_chaz_[2]), np.log(cs_[3,:]/c_s_chaz_[3]), np.log(cs_[4,:]/c_s_chaz_[4])),axis=0)
#x = np.concatenate((xI_[0,:]/R_, xI_[1,:]/R_, xI_[2,:]/R_, xI_[3,:]/R_, xI_[4,:]/R_))

#y = np.concatenate((np.log(cs_[1,:]/c_s_chaz_[1]), np.log(cs_[2,:]/c_s_chaz_[2]), np.log(cs_[3,:]/c_s_chaz_[3]), np.log(cs_[4,:]/c_s_chaz_[4])),axis=0)
#x = np.concatenate((xI_[1,:]/R_, xI_[2,:]/R_, xI_[3,:]/R_, xI_[4,:]/R_))

y = np.concatenate((np.log(cs_[0,1:]/c_s_chaz_[0]), np.log(cs_[1,1:]/c_s_chaz_[1]), np.log(cs_[2,1:]/c_s_chaz_[2]), np.log(cs_[3,1:]/c_s_chaz_[3]), np.log(cs_[4,1:]/c_s_chaz_[4])),axis=0)
x = np.concatenate((xI_[0,1:]/R_[1:], xI_[1,1:]/R_[1:], xI_[2,1:]/R_[1:], xI_[3,1:]/R_[1:], xI_[4,1:]/R_[1:]))

figure()


plot(x,y,'.')

p = np.polyfit(x,y,1)
print (p)

plot(x,np.polyval(p,x),'r')

[-0.33585666 -0.05760673]


# R=f(j) 

In [8]:
j_075 = np.array([2.5, 5, 10])*0.001/(50*1e-06*7.5*0.01)/10  
j_05 = np.array([1.75, 2.5, 5, 10])*0.001/(50*1e-06*7.5*0.01)/10 

Rg_075 = np.array([274.4, 348.6, 208.9])
Rg_05 = np.array([390.9, 435.7, 220.35, 134.9])

R_inc_075 = np.array([75.93, 95.84, 40.17])
R_inc_05 = np.array([164.45, 78.46, 60.28, 52.18])


figure(figsize=(9,8))

plot([82.1, 82.1],[0, 1000],'-k',label=r'$j_c$, $c_0$ = 0.50 M')
plot([92.3, 92.3],[0, 1000],'--k',label=r'$j_c$, $c_0$ = 0.75 M')

plot(500,700,'bs',label=r'$c_0$ = 0.75 M')
plot(500,700,'ro',label=r'$c_0$ = 0.50 M')
errorbar(j_075,Rg_075,yerr=R_inc_075,fmt='bs',capsize=5)
errorbar(j_05,Rg_05,yerr=R_inc_05,fmt='ro',capsize=5)



jjj = np.array([10, 30, 50, 70, 90])
xII = np.array([175, 126, 109, 100, 94])
#plot(jjj,xII,'g')


#for cap in caps:
#    cap.set_color('red')
#    cap.set_markeredgewidth(10)

#plot(j0_exp/10,Rg*1e+09,'-')
xlabel(r'$j$ (mA/cm$^2$)',fontsize=20)
ylabel(r'$R_g$ (nm)',fontsize=20)

xticks(fontsize=15)
yticks(fontsize=15)


ylim(0,650)
xlim(0,350)


# ajustement Rg = a0*(1/j)**n
popt075, pcov = curve_fit(func_adjust_Rg075, j_075, Rg_075,p0=[300., 1.],sigma=R_inc_075)
popt05, pcov = curve_fit(func_adjust_Rg05, j_05[0:-1], Rg_05[0:-1],p0=[300, 1.],sigma=R_inc_05[0:-1])

print (popt075)
print (popt05)


jj = np.linspace(0,150,100)
#plot(jj, func_adjust_Rg075(jj, *popt075),'g')
plot(jj, func_adjust_Rg05(jj, *popt05),'-.r',label=r'$R_g$, $c_0$ = 0.50 M, fitted by ${R_g}_c(j_c/j)^{N}$')
plot(1,1,'.w',label=r' ')
#func_adjust_Rg(x,a0,N)

text(175,415,r'${R_g}_c=333.89$ nm, $N=0.713$',fontsize=16)

legend(loc='upper right',fontsize=18,framealpha=0) 



[2.77984929e+02 2.27157202e-01]
[333.88879166   0.71340853]


/home/fabien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in true_divide


# R_moyen = f(j)

In [16]:
j_075 = np.array([2.5, 5, 10])*0.001/(50*1e-06*7.5*0.01)/10  
j_05 = np.array([1.75, 2.5, 5, 10])*0.001/(50*1e-06*7.5*0.01)/10 

Rg_075 = np.array([274.4, 348.6, 208.9])
Rg_05 = np.array([390.9, 435.7, 220.35, 134.9])

R_inc_075 = np.array([75.93, 95.84, 40.17])
R_inc_05 = np.array([164.45, 78.46, 60.28, 52.18])



j_R = j_05[0:3]

#R_gg = np.array([390.9, (274.4+435.7)/2, (348.6+220.35)/2])
R_gg = np.array([390.9, (274.4+435.7)/2, (444.38+159.58)/2])


R_gg_inc = np.array([164.45, (512.-198.)/2, (444.38-159.58)/2 ])

figure()

errorbar(j_075,Rg_075,yerr=R_inc_075,fmt='bs',capsize=5)
errorbar(j_05,Rg_05,yerr=R_inc_05,fmt='ro',capsize=5)

errorbar(j_R,R_gg,yerr=R_gg_inc,fmt='gs',capsize=5)

ylim(0,650)
xlim(0,350)


(0, 350)

# eta_0 = f(j)

In [134]:
figure(figsize=(9,8))

j = np.array([5., 15, 30., 50., 60., 70., 80., 90., 100.])*10

#xI : [320.517584, 220.4172329, 174.134771, 146.41503172, 137.638260 , 130.632996, 124.856198, 119.97502, 115.772043 ]
#c_s : [0.01850484, 0.0433109, 0.07292425, 0.10634875, 0.121528, 0.1359761, 0.149821, 0.163158247, 0.176056 ]
# eta0 : [0.1339829, 0.1232442, 0.1166647, 0.1119001, 0.11021518, 0.10879664, 0.10757213, 0.106495, 0.10553442 ]

R_ =  333.89*1e-09


vg_025 = (1-tc)*j/(n*F*250.)
jg_025 = (1./6.)*n*F*(rho/M)*vg_025

vg_05 = (1-tc)*j/(n*F*500.)
jg_05 = (1./6.)*n*F*(rho/M)*vg_05

vg_075 = (1-tc)*j/(n*F*750.)
jg_075 = (1./6.)*n*F*(rho/M)*vg_075

xI_025 = np.array([220.41723, 151.742363, 119.975023, 100.93720, 94.906776, 90.0924726, 86.12165,  82.7658755, 79.87590])
#c_s : [0.04331, 0.0984286, 0.163158, 0.2355623, 0.268299,  0.2993957, 0.329146, 0.357764, 0.385410]
# eta0 : [0.1093707, 0.099003, 0.092621724, 0.0879839, 0.0863406,  0.084955, 0.083759, 0.082706,  0.08176665  ]

xI_05 = np.array([279.121878, 192.01885, 151.74236314, 127.6153265, 119.975023, 113.876387, 108.8468904, 104.596900, 100.937206   ])# a calculer cas par cas
#c_s : [0.0254229, 0.05882, 0.098428, 0.14296763, 0.163158, 0.18235929, 0.2007473, 0.21844980, 0.2355623   ]
# eta0 : [0.124851, 0.114258, 0.107757, 0.103043, 0.101374, 0.0999699, 0.098756, 0.09768957, 0.0967371   ]


xI_075 = np.array([320.517584, 220.4172329, 174.134771, 146.41503172, 137.638260 , 130.632996, 124.856198, 119.97502, 115.772043])
#c_s : [0.01850484, 0.0433109, 0.07292425, 0.10634875, 0.121528, 0.1359761, 0.149821, 0.163158247, 0.176056 ]
# eta0 : [0.1339829, 0.1232442, 0.1166647, 0.1119001, 0.11021518, 0.10879664, 0.10757213, 0.106495, 0.10553442 ]

#anciennes valeurs (1/3)
#xI_025 = np.array([174.134771, 119.9750, 94.9067, 79.8759, 75.1131, 71.310, 68.1733, 65.5219, 63.2383])
#xI_05 = np.array([220.41723, 151.7423, 119.9750, 101.10, 95.05, 90.22, 86.24, 82.88, 79.98])# a calculer cas par cas
#xI_075 = np.array([253.05327225, 174.1347710, 137.638260, 115.97,109.03,103.48,98.91,95.05,91.72])


cs_025 = (2*(n*F)**3*uc/(3*jg_025*epss*epss0))**(-0.5)*(((xI_025*1e-09+R_)**3-R_**3)/R_**2)**(-0.5)
cs_05 = (2*(n*F)**3*uc/(3*jg_05*epss*epss0))**(-0.5)*(((xI_05*1e-09+R_)**3-R_**3)/R_**2)**(-0.5)
cs_075 = (2*(n*F)**3*uc/(3*jg_075*epss*epss0))**(-0.5)*(((xI_075*1e-09+R_)**3-R_**3)/R_**2)**(-0.5)


eta0_025 = (RR*TT/(n*F))*np.log(cs_025/250.)
eta0_05 = (RR*TT/(n*F))*np.log(cs_05/500.)
eta0_075 = (RR*TT/(n*F))*np.log(cs_075/750.)

plot(j/10,-eta0_025,'-kv',label=r'$c_0$ = 0.25 M')
plot(j/10,-eta0_05,'-bs',label=r'$c_0$ = 0.50 M')
plot(j/10,-eta0_075,'-ro',label=r'$c_0$ = 0.75 M')

legend(loc='lower left',fontsize=18) 

xlabel(r'$j$ (mA/cm$^2$)',fontsize=20)
ylabel(r'$-\eta_0$ (V)',fontsize=20)

xticks(fontsize=15)
yticks(fontsize=15)

#ylim(0.07,0.115)
#xlim(0,350)


(array([0.07, 0.08, 0.09, 0.1 , 0.11, 0.12, 0.13, 0.14]),
 <a list of 8 Text yticklabel objects>)

# R_t standard = f(j)

In [77]:
#j = np.array([40., 60., 80., 100., 120., ])


j_075 = np.array([2.5, 5, 10])*0.001/(50*1e-06*7.5*0.01)/10  
j_05 = np.array([1.75, 2.5, 5, 10])*0.001/(50*1e-06*7.5*0.01)/10 

Rg_075 = np.array([274.4, 348.6, 208.9])
Rg_05 = np.array([390.9, 435.7, 220.35, 134.9])

R_inc_075 = np.array([75.93, 95.84, 40.17])
R_inc_05 = np.array([164.45, 78.46, 60.28, 52.18])



j_R = j_05[0:3]

j = j_R*10

#R_gg = np.array([390.9, (274.4+435.7)/2, (348.6+220.35)/2])
R_gg = np.array([390.9, (274.4+435.7)/2, (444.38+159.58)/2])
R_gg_inc = np.array([164.45, (512.-198.)/2, (444.38-159.58)/2 ])





l = 2.

tau = gamma*vm/(RR*TT)


vg_05_ = (1-tc)*j/(n*F*500.)
jg_05_ = (1./6.)*n*F*(rho/M)*vg_05_
xI_chaz_05_ = 1e+09*((3./2.)*dV)**(2/3.)*(epss*epss0*n*F*uc/(2*jg_05_))**(1/3.)  
c_s_chaz_05_ = (jg_05_*epss*epss0/(2*n*F*n*uc*F*F*n*1e-09*xI_chaz_05_))**0.5

cs_05_ = np.exp(-0.336*xI_chaz_05_/R_gg)*c_s_chaz_05_
R_c_stand_05 = l*(l+1)*tau/((500.-cs_05_)/cs_05_)

cs_05_ = np.exp(-0.336*xI_chaz_05_/(R_gg+R_gg_inc))*c_s_chaz_05_
R_c_stand_05_sup = l*(l+1)*tau/((500.-cs_05_)/cs_05_)

cs_05_ = np.exp(-0.336*xI_chaz_05_/(R_gg-R_gg_inc))*c_s_chaz_05_
R_c_stand_05_inf = l*(l+1)*tau/((500.-cs_05_)/cs_05_)

err_05 = np.array([R_c_stand_05-R_c_stand_05_inf, -R_c_stand_05+R_c_stand_05_sup])


#########

vg_075_ = (1-tc)*j/(n*F*750.)
jg_075_ = (1./6.)*n*F*(rho/M)*vg_075_
xI_chaz_075_ = 1e+09*((3./2.)*dV)**(2/3.)*(epss*epss0*n*F*uc/(2*jg_075_))**(1/3.)  
c_s_chaz_075_ = (jg_075_*epss*epss0/(2*n*F*n*uc*F*F*n*1e-09*xI_chaz_075_))**0.5

cs_075_ = np.exp(-0.336*xI_chaz_075_/R_gg)*c_s_chaz_075_
R_c_stand_075 = l*(l+1)*tau/((750.-cs_075_)/cs_075_)

cs_075_ = np.exp(-0.336*xI_chaz_075_/(R_gg+R_gg_inc))*c_s_chaz_075_
R_c_stand_075_sup = l*(l+1)*tau/((750.-cs_075_)/cs_075_)

cs_075_ = np.exp(-0.336*xI_chaz_075_/(R_gg-R_gg_inc))*c_s_chaz_075_
R_c_stand_075_inf = l*(l+1)*tau/((750.-cs_075_)/cs_075_)

err_075 = np.array([R_c_stand_075-R_c_stand_075_inf, -R_c_stand_075+R_c_stand_075_sup])

#####
#figure()
#plot(j/10,(((500.-cs_05_)/cs_05_)*333.89/(tau*1e+09))**0.5)
#print(2*pi*333.089/(tau*1e+09*483))

#####
jj = np.linspace(0,82.1,200)
Rg_fitted = 333.89*(82.1/jj)**0.713
#####

ll=arange(300.,1000.,100.)

RR_ = ll*(1.16*tau*1e+09)/(2*np.pi)
jj_ = (333.89/RR_)**(1/0.713)*82.1

j_c0 = jj_*10/500.
vg_ = (1-tc)*j_c0/(n*F)
jg_ = (1./6.)*n*F*(rho/M)*vg_
xI = 1e+09*((3./2.)*dV)**(2/3.)*(epss*epss0*n*F*uc/(2*jg_))**(1/3.)

c_s_chaz_05_ll = (jg_*epss*epss0/(2*n*F*n*uc*F*F*n*1e-09*xI))**0.5

cs_05_ll = np.exp(-0.336*xI/RR_)*c_s_chaz_05_ll
R_c_stand_05_ll = ll*(ll+1)*tau/((500.-cs_05_ll)/cs_05_ll)

###########




figure(figsize=(9,8))

errorbar(j_05,Rg_05,yerr=R_inc_05,fmt='ro',capsize=5,label=r'$R_g$, $c_0$ = 0.50 M')
errorbar(j_075,Rg_075,yerr=R_inc_075,fmt='bs',capsize=5,label=r'$R_g$, $c_0$ = 0.75 M')

errorbar(j/10,1e+09*R_c_stand_05,yerr=1e+09*err_05,fmt='-xb',capsize=5,label=r'$\mathrm{min}(R_t)$, $c_0$ = 0.50 M')
errorbar(j/10,1e+09*R_c_stand_075,yerr=1e+09*err_075,fmt='v-r',capsize=5,label=r'$\mathrm{min}(R_t)$, $c_0$ = 0.75 M')

#plot(jj_,1e+09*R_c_stand_05_ll,'ob')
#for i in range(jj_.size):
#    text(jj_[i]+1,1e+09*R_c_stand_05_ll[i],str(int(ll[i])))

#plot(jj,Rg_fitted,'-.r',label=r'$R_g = 333.89 (j_c(0.5 \ \mathrm{M})/j)^{0.713}$')

plot([82.1, 82.1],[0.001, 5000],'-k',label=r'$j_c$, $c_0$ = 0.50 M')
plot([92.3, 92.3],[0.001, 5000],'--k',label=r'$j_c$, $c_0$ = 0.75 M')

ylim(0.003,600)
xlim(30,150)

yscale('log')


xlabel(r'$j$ (mA/cm$^2$)',fontsize=20)
ylabel(r'Particle size (nm)',fontsize=20)

xticks(fontsize=15)
yticks(fontsize=15)

legend(loc='center right',fontsize=15) 


/home/fabien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:74: RuntimeWarning: divide by zero encountered in true_divide



# Rt = xI(T) 

In [10]:
l = 2.







Rt_05_=xI_chaz_05_+2*1e+09*tau*(l+2)*(l-1)

Rt_05_2 = xI_chaz_05_*0.5*(1+((1-2*tau/(xI_chaz_05_*1e-09))**2+8*(1+(l+2)*(l-1)*(tau/(1e-09*xI_chaz_05_))))**0.5)-tau

Rt_075_=xI_chaz_075_+2*1e+09*tau*(l+2)*(l-1)

print (tau)

figure(figsize=(9,8))


plot(j/10,Rt_05_ ,'-xb')
plot(j/10,Rt_05_2 ,'-xg')
plot(j/10,Rt_075_ ,'v-r')

errorbar(j_05,Rg_05,yerr=R_inc_05,fmt='ro',capsize=5,label=r'$R_g$, $c_0$ = 0.50 M')
errorbar(j_075,Rg_075,yerr=R_inc_075,fmt='bs',capsize=5,label=r'$R_g$, $c_0$ = 0.75 M')

plot([82.1, 82.1],[0.001, 5000],'-k',label=r'$j_c$, $c_0$ = 0.50 M')
plot([92.3, 92.3],[0.001, 5000],'--k',label=r'$j_c$, $c_0$ = 0.75 M')

ylim(0,650)
xlim(25,175)

NameError: name 'xI_chaz_05_' is not defined

# Rt = xI(T) II (premier ordre en delta uniquement)

In [51]:
l = 8.




Rt_05_=xI_chaz_05_*0.5*(1+((1-2*tau/(xI_chaz_05_*1e-09))**2+8*(1+(l+2)*(l-1)*(tau/(1e-09*xI_chaz_05_))))**0.5)-tau
Rt_075_=xI_chaz_075_*0.5*(1+((1-2*tau/(xI_chaz_075_*1e-09))**2+8*(1+(l+2)*(l-1)*(tau/(1e-09*xI_chaz_075_))))**0.5)-tau

#l=2.
l = (2*np.pi*323*(80/66.67))/(60.13*tau*1e+09)
Rt_05_l1=xI_chaz_05_*0.5*(1+((1-2*tau/(xI_chaz_05_*1e-09))**2+8*(1+(l+2)*(l-1)*(tau/(1e-09*xI_chaz_05_))))**0.5)-tau
Rt_075_l1=xI_chaz_075_*0.5*(1+((1-2*tau/(xI_chaz_075_*1e-09))**2+8*(1+(l+2)*(l-1)*(tau/(1e-09*xI_chaz_075_))))**0.5)-tau


#l=20.
l = (2*np.pi*323*(80/46.67))/(60.13*tau*1e+09)
Rt_05_l2=xI_chaz_05_*0.5*(1+((1-2*tau/(xI_chaz_05_*1e-09))**2+8*(1+(l+2)*(l-1)*(tau/(1e-09*xI_chaz_05_))))**0.5)-tau
Rt_075_l2=xI_chaz_075_*0.5*(1+((1-2*tau/(xI_chaz_075_*1e-09))**2+8*(1+(l+2)*(l-1)*(tau/(1e-09*xI_chaz_075_))))**0.5)-tau


popt05, pcov = curve_fit(func_adjust_Rg05, j_05[0:-1], Rg_05[0:-1],p0=[300, 1.],sigma=R_inc_05[0:-1])
print (popt05)
jj = np.linspace(0,160,100)

ll=np.array([4,5,6,7,8,9.,10.,11.,12,13,14,15.,16,17.])


vg__ = (1-tc)*82.1*10/(n*F*500.)
jg__ = (1./6.)*n*F*(rho/M)*vg__
xI__ = 1e+09*((3./2.)*dV)**(2/3.)*(epss*epss0*n*F*uc/(2*(jg__)))**(1/3.)  
RR_ll__=xI__*0.5*(1+((1-2*tau/(xI__*1e-09))**2+8*(1+(ll+2)*(ll-1)*(tau/(1e-09*xI__))))**0.5)-tau
#plot(ll,RR_ll__,'.')
# l=8 pour Rt = Rg = 333.89 nm

alphaa = 2*np.pi*333.89*1e-09/(tau*8)
print (alphaa)
#alphaa = 70.00

RR_ = ll*(70.*tau*1e+09)/(2*np.pi)
jj_ = (333.89/RR_)**(1/0.713)*82.1

j_c0 = jj_*10/500.
vg_ = (1-tc)*j_c0/(n*F)
jg_ = (1./6.)*n*F*(rho/M)*vg_
xI = 1e+09*((3./2.)*dV)**(2/3.)*(epss*epss0*n*F*uc/(2*jg_))**(1/3.)  

Rt_05_ll=xI*0.5*(1+((1-2*tau/(xI*1e-09))**2+8*(1+(ll+2)*(ll-1)*(tau/(1e-09*xI))))**0.5)-tau


l = 8.
j_c0 = jj*10/500.
vg_ = (1-tc)*j_c0/(n*F)
jg_ = (1./6.)*n*F*(rho/M)*vg_
xI = 1e+09*((3./2.)*dV)**(2/3.)*(epss*epss0*n*F*uc/(2*jg_))**(1/3.)  
Rt_05_ll_simple=xI*0.5*(1+((1-2*tau/(xI*1e-09))**2+8*(1+(l+2)*(l-1)*(tau/(1e-09*xI))))**0.5)-tau

l = 8.
j_c0 = jj*10/750.
vg_ = (1-tc)*j_c0/(n*F)
jg_ = (1./6.)*n*F*(rho/M)*vg_
xI = 1e+09*((3./2.)*dV)**(2/3.)*(epss*epss0*n*F*uc/(2*jg_))**(1/3.)  
Rt_075_ll_simple=xI*0.5*(1+((1-2*tau/(xI*1e-09))**2+8*(1+(l+2)*(l-1)*(tau/(1e-09*xI))))**0.5)-tau

Rg05_fitted = func_adjust_Rg05(jj, *popt05)



print (tau, xI_chaz_05_,j)
print (2*np.pi*200*1e-09/(2*tau)/(np.pi*6**0.5))



figure(figsize=(9,8))

errorbar(j_05,Rg_05,yerr=R_inc_05,fmt='ro',capsize=5,label=r'$R_g$, $c_0$ = 0.50 M')
errorbar(j_075,Rg_075,yerr=R_inc_075,fmt='bs',capsize=5,label=r'$R_g$, $c_0$ = 0.75 M')

plot(jj,Rt_05_ll_simple,'-r',label=r'$\mathrm{max}(R_t(8))$, $c_0$ = 0.50 M')
plot(jj_,Rt_05_ll,'Dr',label=r'$\mathrm{max}(R_t(l_M))$, $c_0$ = 0.50 M')
plot(jj,Rt_075_ll_simple,'-b',label=r'$\mathrm{max}(R_t(8))$, $c_0$ = 0.75 M')

for i in range(jj_.size):
    text(jj_[i]+1,Rt_05_ll[i]+10,str(int(ll[i])))

plot(jj[0:52], func_adjust_Rg05(jj[0:52], *popt05),'-.r',label=r'$R_g = 333.89 (j_c(0.5 \ \mathrm{M})/j)^{0.713}$')


#plot(j/10,Rt_05_ ,'-xb')
#plot(j/10,Rt_075_ ,'v-r')

#plot(j/10,np.array([Rt_05_l2[0], Rt_05_l1[1], Rt_05_[2]]),'--b')
#plot(j/10,np.array([Rt_075_l2[0], Rt_075_l1[1], Rt_075_[2]]),'--r')

#plot(jj, func_adjust_Rg05(jj, *popt05),'-.r',label=r'$R_g$, $c_0$ = 0.50 M, fitted by $333.89(j_c/j)^{0.713}$')




plot([82.1, 82.1],[0.001, 5000],'-k',label=r'$j_c$, $c_0$ = 0.50 M')
plot([92.3, 92.3],[0.001, 5000],'--k',label=r'$j_c$, $c_0$ = 0.75 M')

ylim(0,800)
xlim(25,165)



xlabel(r'$j$ (mA/cm$^2$)',fontsize=20)
ylabel(r'Particle size (nm)',fontsize=20)

xticks(fontsize=15)
yticks(fontsize=15)

legend(loc='upper right',fontsize=15) 

#figure()
#plot(jj, func_adjust_Rg05(jj, *popt05),'-.b',label=r'$R_g = 333.89 (j_c(0.5 \ \mathrm{M})/j)^{0.713}$')
#plot(jj_,Rt_05_ll,'Dr',label=r'$\mathrm{max}(R_t(l_M))$, $c_0$ = 0.50 M')



[333.88879166   0.71340853]
69.99823033382019
3.7463317504767314e-09 [127.98171797 113.63547953  90.19253987] [ 466.66666667  666.66666667 1333.33333333]
21.794561595454663


/home/fabien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in true_divide
/home/fabien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: RuntimeWarning: divide by zero encountered in true_divide
/home/fabien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in true_divide
/home/fabien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in true_divide


In [56]:
#inset
figure(figsize=(9,8))
plot(jj[0:52], func_adjust_Rg05(jj[0:52], *popt05),'-.k',label=r'$R_g = 333.89 (j_c/j)^{0.713}$')
plot(jj_[4::],Rt_05_ll[4::],'Dr',label=r'$R_t(l_M)$, $c_0$ = 0.50 M')

for i in range(jj_[3::].size):
    text(jj_[i+3]+1,Rt_05_ll[i+3]+10,str(int(ll[i+3])),fontsize=25)

ylim(200,750)
xlim(25,100)



xlabel(r'$j$ (mA/cm$^2$)',fontsize=30)
ylabel(r'$R$ (nm)',fontsize=30)

xticks(fontsize=25)
yticks(fontsize=25)
legend(loc='upper right',fontsize=25) 

/home/fabien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in true_divide


## Avec Rg_fitted

In [29]:
l = 8.




Rt_05_=xI_chaz_05_*0.5*(1+((1-2*tau/(xI_chaz_05_*1e-09))**2+8*(1+(l+2)*(l-1)*(tau/(1e-09*xI_chaz_05_))))**0.5)-tau
Rt_075_=xI_chaz_075_*0.5*(1+((1-2*tau/(xI_chaz_075_*1e-09))**2+8*(1+(l+2)*(l-1)*(tau/(1e-09*xI_chaz_075_))))**0.5)-tau

#l=2.
l = (2*np.pi*323*(80/66.67))/(60.13*tau*1e+09)
Rt_05_l1=xI_chaz_05_*0.5*(1+((1-2*tau/(xI_chaz_05_*1e-09))**2+8*(1+(l+2)*(l-1)*(tau/(1e-09*xI_chaz_05_))))**0.5)-tau
Rt_075_l1=xI_chaz_075_*0.5*(1+((1-2*tau/(xI_chaz_075_*1e-09))**2+8*(1+(l+2)*(l-1)*(tau/(1e-09*xI_chaz_075_))))**0.5)-tau


#l=20.
l = (2*np.pi*323*(80/46.67))/(60.13*tau*1e+09)
Rt_05_l2=xI_chaz_05_*0.5*(1+((1-2*tau/(xI_chaz_05_*1e-09))**2+8*(1+(l+2)*(l-1)*(tau/(1e-09*xI_chaz_05_))))**0.5)-tau
Rt_075_l2=xI_chaz_075_*0.5*(1+((1-2*tau/(xI_chaz_075_*1e-09))**2+8*(1+(l+2)*(l-1)*(tau/(1e-09*xI_chaz_075_))))**0.5)-tau





popt05, pcov = curve_fit(func_adjust_Rg05, j_05[0:-1], Rg_05[0:-1],p0=[300, 1.],sigma=R_inc_05[0:-1])
print (popt05)
jj = np.linspace(0,160,100)

ll=np.array([4,5,6,7,8,9.,10.,11.,12,13,14,15.,16,17.])


figure()
vg__ = (1-tc)*82.1*10/(n*F*500.)
jg__ = (1./6.)*n*F*(rho/M)*vg__
xI__ = 1e+09*((3./2.)*dV)**(2/3.)*(epss*epss0*n*F*uc/(2*(jg__)))**(1/3.)  
RR_ll__=xI__*0.5*(1+((1-2*tau/(xI__*1e-09))**2+8*(1+(ll+2)*(ll-1)*(tau/(1e-09*xI__))))**0.5)-tau
plot(ll,RR_ll__,'.')
# l=8 pour Rt = Rg = 333.89 nm

alphaa = 2*np.pi*333.89*1e-09/(tau*8)
print (alphaa)
#alphaa = 70.00

RR_ = ll*(70.*tau*1e+09)/(2*np.pi)
jj_ = (333.89/RR_)**(1/0.713)*82.1

j_c0 = jj_*10/500.
vg_ = (1-tc)*j_c0/(n*F)
jg_ = (1./6.)*n*F*(rho/M)*vg_
xI = 1e+09*((3./2.)*dV)**(2/3.)*(epss*epss0*n*F*uc/(2*jg_))**(1/3.)  

Rt_05_ll=xI*0.5*(1+((1-2*tau/(xI*1e-09))**2+8*(1+(ll+2)*(ll-1)*(tau/(1e-09*xI))))**0.5)-tau


l = 8.
j_c0 = jj*10/500.
vg_ = (1-tc)*j_c0/(n*F)
jg_ = (1./6.)*n*F*(rho/M)*vg_
xI = 1e+09*((3./2.)*dV)**(2/3.)*(epss*epss0*n*F*uc/(2*jg_))**(1/3.)  
Rt_05_ll_simple=xI*0.5*(1+((1-2*tau/(xI*1e-09))**2+8*(1+(l+2)*(l-1)*(tau/(1e-09*xI))))**0.5)-tau

l = 8.
j_c0 = jj*10/750.
vg_ = (1-tc)*j_c0/(n*F)
jg_ = (1./6.)*n*F*(rho/M)*vg_
xI = 1e+09*((3./2.)*dV)**(2/3.)*(epss*epss0*n*F*uc/(2*jg_))**(1/3.)  
Rt_075_ll_simple=xI*0.5*(1+((1-2*tau/(xI*1e-09))**2+8*(1+(l+2)*(l-1)*(tau/(1e-09*xI))))**0.5)-tau

Rg05_fitted = func_adjust_Rg05(jj, *popt05)



print (tau, xI_chaz_05_,j)
print (2*np.pi*200*1e-09/(2*tau)/(np.pi*6**0.5))



figure(figsize=(9,8))

errorbar(j_05,Rg_05,yerr=R_inc_05,fmt='ro',capsize=5,label=r'$R_g$, $c_0$ = 0.50 M')
errorbar(j_075,Rg_075,yerr=R_inc_075,fmt='bs',capsize=5,label=r'$R_g$, $c_0$ = 0.75 M')

plot(jj,Rt_05_ll_simple,'-r',label=r'$R_t(l=8)$, $c_0$ = 0.50 M')
plot(jj_,Rt_05_ll,'Dr',label=r'$R_t(l_M)$, $c_0$ = 0.50 M')
plot(jj,Rt_075_ll_simple,'-b',label=r'$R_t(l=8)$, $c_0$ = 0.75 M')

for i in range(jj_.size):
    text(jj_[i]+1,Rt_05_ll[i]+10,str(int(ll[i])))


jj = np.linspace(0,150,100)
#plot(jj, func_adjust_Rg075(jj, *popt075),'g')
plot(jj, func_adjust_Rg05(jj, *popt05),'-.r',label=r'$R_g$ fitted by ${R_g}_c(j_c/j)^{N}$')
#plot(j/10,Rt_05_ ,'-xb')
#plot(j/10,Rt_075_ ,'v-r')

#plot(j/10,np.array([Rt_05_l2[0], Rt_05_l1[1], Rt_05_[2]]),'--b')
#plot(j/10,np.array([Rt_075_l2[0], Rt_075_l1[1], Rt_075_[2]]),'--r')

#plot(jj, func_adjust_Rg05(jj, *popt05),'-.r',label=r'$R_g$, $c_0$ = 0.50 M, fitted by $333.89(j_c/j)^{0.713}$')




plot([82.1, 82.1],[0.001, 5000],'-k',label=r'$j_c$, $c_0$ = 0.50 M')
plot([92.3, 92.3],[0.001, 5000],'--k',label=r'$j_c$, $c_0$ = 0.75 M')

ylim(0,750)
xlim(25,165)



xlabel(r'$j$ (mA/cm$^2$)',fontsize=20)
ylabel(r'$R$ (nm)',fontsize=20)

xticks(fontsize=15)
yticks(fontsize=15)

legend(loc='upper right',fontsize=15) 


figure()
plot(jj_,Rt_05_ll-func_adjust_Rg05(jj_, *popt05),'r',label=r'$R_t(l_M)$, $c_0$ = 0.50 M')

[333.88879166   0.71340853]
69.99823033382019
3.7463317504767314e-09 [127.98171797 113.63547953  90.19253987] [ 466.66666667  666.66666667 1333.33333333]
21.794561595454663


/home/fabien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: divide by zero encountered in true_divide
/home/fabien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in true_divide
/home/fabien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in true_divide
/home/fabien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in true_divide


In [14]:
ll = np.linspace(0,30,100)

RRRtt = 92.5*1e-09*0.5*(1+((1-2*tau/(92.5*1e-09))**2+8*(1+(ll+2)*(ll-1)*(tau/(92.5*1e-09))))**0.5)-tau

figure()

plot(ll,RRRtt)
plot([0, 30],[323*1e-09, 323*1e-09])

print 'alpha=', 2*np.pi*323*1e-09/(3.75*1e-09*9)

alpha= 60.1324104954


# comp termes dV

In [24]:
j_c0 = np.array([0.4, 3.05, 5.7, 8.35, 11.])

R_ = np.array([25., 50., 100., 150., 250., 350., 450., 550])

xI_ = np.zeros((5,8))
cs_ = np.zeros((5,8))

#xI_chaz = np.zeros((5,8))
#cs_chaz = np.zeros((5,8))


xI_[0,:]=[145.50, 143.62, 141.28, 139.91, 138.37, 137.53, 137.01, 136.64]
xI_[1,:]=[73.00, 71.81, 70.65,70.08, 69.52, 69.25,  69.09, 68.98]
xI_[2,:]=[58.98, 58.00, 57.11, 56.69, 56.30, 56.11, 56.00, 55.93]
xI_[3,:]=[51.78, 50.91, 50.16, 49.82, 49.50,  49.35, 49.26, 49.21]
xI_[4,:]=[47.13, 46.34, 45.68, 45.39, 45.12, 44.99, 44.92, 44.87]

cs_[0,:]=[0.0337, 0.0560, 0.0829, 0.09799, 0.1142, 0.1226, 0.128, 0.1312]
cs_[1,:]=[0.2147, 0.3185, 0.4148, 0.4596, 0.5012, 0.5211, 0.5327, 0.5403]
cs_[2,:]=[0.3719, 0.5303, 0.666, 0.7256, 0.7800, 0.8054, 0.8201, 0.8297]
cs_[3,:]=[0.5171, 0.7196, 0.8853, 0.9560, 1.0196,  1.0490, 1.0660, 1.0770]
cs_[4,:]=[0.6543, 0.8947, 1.0852, 1.1650, 1.2360, 1.2686, 1.2873, 1.299]





vg_ = (1-tc)*j_c0/(n*F)
jg_ = (1./3.)*n*F*(rho/M)*vg_
xI_chaz_ = 1e+09*((3./2.)*dV)**(2/3.)*(epss*epss0*n*F*uc/(2*jg_))**(1/3.)  
c_s_chaz_ = (jg_*epss*epss0/(2*n*F*n*uc*F*F*n*1e-09*xI_chaz_))**0.5

term_sph = np.zeros((j_c0.size,R_.size))

for j in range(R_.size):
    for i in range(j_c0.size):
        x = 1e-09*np.arange(R_[j],R_[j]+xI_[i,j],xI_[i,j]/1000.)
        y = ((1e-09*xI_[i,j]+1e-09*R_[j])**3-x**3)**0.5/x**2
        term_sph[i,j] = 1e-09*R_[j]*simps(y,x)*(2*jg_[i]/(3*n*F*uc*epss*epss0))**0.5
        
term_1D = (2./3.)*(2*jg_/(epss*epss0*n*F*uc))**0.5*(xI_chaz_*1e-09)**(3./2.)

figure()
plot(j_c0, term_1D)
plot(j_c0, term_sph[:,0])
plot(j_c0, term_sph[:,1])
plot(j_c0, term_sph[:,2])
plot(j_c0, term_sph[:,3])
plot(j_c0, term_sph[:,4])
plot(j_c0, term_sph[:,5])
plot(j_c0, term_sph[:,6])

ylim(0.9, 1.1)


/home/fabien/.local/lib/python2.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in sqrt


(0.9, 1.1)

In [7]:
j_c0 = np.array([0.4, 3.05, 5.7, 8.35, 11.])

R_ = np.array([25., 50., 100., 150., 250., 350., 450., 550])

xI_ = np.zeros((5,8))
cs_ = np.zeros((5,8))

#xI_chaz = np.zeros((5,8))
#cs_chaz = np.zeros((5,8))


xI_[0,:]=[145.50, 143.62, 141.28, 139.91, 138.37, 137.53, 137.01, 136.64]
xI_[1,:]=[73.00, 71.81, 70.65,70.08, 69.52, 69.25,  69.09, 68.98]
xI_[2,:]=[58.98, 58.00, 57.11, 56.69, 56.30, 56.11, 56.00, 55.93]
xI_[3,:]=[51.78, 50.91, 50.16, 49.82, 49.50,  49.35, 49.26, 49.21]
xI_[4,:]=[47.13, 46.34, 45.68, 45.39, 45.12, 44.99, 44.92, 44.87]

cs_[0,:]=[0.0337, 0.0560, 0.0829, 0.09799, 0.1142, 0.1226, 0.128, 0.1312]
cs_[1,:]=[0.2147, 0.3185, 0.4148, 0.4596, 0.5012, 0.5211, 0.5327, 0.5403]
cs_[2,:]=[0.3719, 0.5303, 0.666, 0.7256, 0.7800, 0.8054, 0.8201, 0.8297]
cs_[3,:]=[0.5171, 0.7196, 0.8853, 0.9560, 1.0196,  1.0490, 1.0660, 1.0770]
cs_[4,:]=[0.6543, 0.8947, 1.0852, 1.1650, 1.2360, 1.2686, 1.2873, 1.299]





vg_ = (1-tc)*j_c0/(n*F)
jg_ = (1./3.)*n*F*(rho/M)*vg_
xI_chaz_ = 1e+09*((3./2.)*dV)**(2/3.)*(epss*epss0*n*F*uc/(2*jg_))**(1/3.)  
c_s_chaz_ = (jg_*epss*epss0/(2*n*F*n*uc*F*F*n*1e-09*xI_chaz_))**0.5

term_sph = np.zeros((j_c0.size,R_.size))

for j in range(R_.size):
    for i in range(j_c0.size):
        x = 1e-09*np.arange(R_[j],R_[j]+xI_[i,j],xI_[i,j]/1000.)
        y = ((1e-09*xI_[i,j]+1e-09*R_[j])**3-x**3)**0.5/x**2
        term_sph[i,j] = 1e-09*R_[j]*simps(y,x)/((1e-09*xI_[i,j])**(3./2.))

print term_sph - 2./(3.)**0.5       

[[-0.12579473 -0.10552663 -0.07939372 -0.06349297         nan -0.03509127
  -0.02866043 -0.02421762]
 [        nan -0.08004239 -0.05341007 -0.04000309 -0.02660648 -0.01992901
          nan         nan]
 [-0.09869198 -0.07178752 -0.04621522 -0.03404693 -0.02225536 -0.0165307
  -0.01317687 -0.01094429]
 [-0.09395558 -0.06679949 -0.04214444 -0.03074444 -0.01994152         nan
  -0.01171515         nan]
 [-0.09044721 -0.06325419         nan -0.0285173  -0.01836223 -0.01354289
  -0.0107283  -0.00888156]]


/home/fabien/.local/lib/python2.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in sqrt
